# Create a Terrace Defect on a Slab

Create a terrace defect by adding layers to a portion of a slab, defined by a cutting plane and number of additional layers.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete.
1. Scroll down to view results.

## Notes

1. The input material must be a Slab, or slab will be generated with provided parameters.

## 1. Prepare the Environment
### 1.1. Set up the notebook
The cut direction, pivot coordinate, and number of added layers define the terrace defect, shown in the image below.

<img src="https://github.com/Exabyte-io/documentation/raw/88451ce38b0f57f804dcb5010a1a95bb9b9e50bc/images/notebooks/terrace_settings.webp" alt="Terrace parameters visualized" width="400"/>

In [1]:
# Material selection
MATERIAL_INDEX = 0  # Which material to use from input list

# Terrace parameters
CUT_DIRECTION = [1, 0, 0]  # Normal vector describing a plane that cuts the terrace from added layers (Miller indices)
PIVOT_COORDINATE = [0.5, 0.5, 0.5]  # Point the cutting plane passes through, in crystal coordinates
NUMBER_OF_ADDED_LAYERS = 1  # Height of terrace in atomic layers
USE_CARTESIAN_COORDINATES = False  # Use cartesian instead of crystal coordinates
ROTATE_TO_MATCH_PBC = True  # Rotate to match periodic boundary conditions

# Slab parameters for creating a new slab if provided material is not a slab
DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0, 0, 1),
    "thickness": 6,
    "vacuum": 10.0,
    "use_orthogonal_z": True,
    "xy_supercell_matrix": [[5, 0], [0, 5]]
}

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [3, 3, 1]  # Structure repeat in view

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [2]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("")

### 1.3. Load input material

In [3]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

0: 0-Ni
1: 1-Graphene
2: C2(001)-Ni4(111), Interface
3: Ni26 C1, Substitution C Defect
4: Ni4(001), termination Ni_P4:mmm_2, Slab
5: O8Zr4(011), termination O2_P2:m_1, Slab
Retrieved 6 materials.


### 1.4. Create a slab if the input material is not a slab

In [4]:
from mat3ra.made.tools.build.slab import create_slab_if_not, SlabConfiguration

default_slab_config = SlabConfiguration(
    bulk=materials[0],
    miller_indices=DEFAULT_SLAB_PARAMETERS["miller_indices"],
    number_of_layers=DEFAULT_SLAB_PARAMETERS["thickness"],
    vacuum=DEFAULT_SLAB_PARAMETERS["vacuum"],
    use_orthogonal_z=DEFAULT_SLAB_PARAMETERS["use_orthogonal_z"],
    xy_supercell_matrix=DEFAULT_SLAB_PARAMETERS["xy_supercell_matrix"]
)

slab = create_slab_if_not(materials[0], default_slab_config)

The material is not a slab. Creating a new slab...


/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['international']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface 

### 1.5. Visualize slab

In [5]:
from utils.visualize import visualize_materials

if SHOW_INTERMEDIATE_STEPS:
    print("Initial slab structure:")
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
    visualize_materials(slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

Initial slab structure:


GridBox(children=(VBox(children=(Label(value='Ni600 - Material - rotation: 0x,0y,0z', layout=Layout(align_self…

GridBox(children=(VBox(children=(Label(value='Ni600 - Material - rotation: -90x', layout=Layout(align_self='ce…

## 2. Create target material
### 2.1. Set up terrace configuration and builder


In [6]:
from mat3ra.made.tools.build.defect.builders import TerraceSlabDefectConfiguration, TerraceSlabDefectBuilder

config = TerraceSlabDefectConfiguration(
    crystal=slab,
    cut_direction=CUT_DIRECTION,
    pivot_coordinate=PIVOT_COORDINATE,
    number_of_added_layers=NUMBER_OF_ADDED_LAYERS,
    use_cartesian_coordinates=USE_CARTESIAN_COORDINATES,
    rotate_to_match_pbc=ROTATE_TO_MATCH_PBC
)

builder = TerraceSlabDefectBuilder()

### 2.2. Generate terrace defect


In [7]:
terrace_slab = builder.get_material(config)

print("\nTerrace defect created with:")
print(f"Cut direction: {CUT_DIRECTION}")
print(f"Pivot point: {PIVOT_COORDINATE}")
print(f"Added layers: {NUMBER_OF_ADDED_LAYERS}")
print(f"Number of atoms: {len(terrace_slab.basis.elements.ids)}")


Terrace defect created with:
Cut direction: [1, 0, 0]
Pivot point: [0.5, 0.5, 0.5]
Added layers: 1
Number of atoms: 654


## 3. Visualize the result

In [8]:
print("Final structure with terrace:")
visualize_materials(terrace_slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)
visualize_materials(terrace_slab, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION, rotation="-90x")

Final structure with terrace:


GridBox(children=(VBox(children=(Label(value='Ni654 - Material - rotation: 0x,0y,0z', layout=Layout(align_self…

GridBox(children=(VBox(children=(Label(value='Ni654 - Material - rotation: -90x', layout=Layout(align_self='ce…

## 4. Pass data to the outside runtime


In [9]:
from utils.jupyterlite import set_materials

set_materials(terrace_slab)

Data for materials written to uploads/Ni4(001), termination Ni_P4:mmm_2, Slab, 1-step Terrace [1, 0, 0].json
